# PLN con Redes Neuronales

In [1]:
import re
import pandas as pd
import numpy as np
import nltk
from nltk import *

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score

from sklearn.preprocessing import MinMaxScaler

### Leyendo el corpus A

In [2]:
corpus_train_esA = pd.read_csv('train_es.tsv',delimiter='\t',encoding='utf-8')
corpus_dev_esA = pd.read_csv('dev_es.tsv',delimiter='\t',encoding='utf-8')

In [3]:
corpus_train_esA

,id,text,HS,TR,AG
0,20001,Easyjet quiere duplicar el número de mujeres p...,1,0,0
1,20002,El gobierno debe crear un control estricto de ...,1,0,0
2,20003,Yo veo a mujeres destruidas por acoso laboral ...,0,0,0
3,20004,"— Yo soy respetuoso con los demás, sólamente l...",0,0,0
4,20007,Antonio Caballero y como ser de mal gusto e ig...,0,0,0
...,...,...,...,...,...
4464,24996,@miriaan_ac @Linaveso_2105 @HumildesSquad_ CÁL...,1,1,1
4465,24997,"@IvanDuque presidente en Cúcuta , tenemos prob...",1,0,1
4466,24998,- Callaté Visto Que Te Dejo En Puta🎤🎶,0,0,0
4467,24999,-¿porque los hombres se casan con las mujeres?...,1,0,0


### Limpieza en los datos
* Cambiar todas las palabras de mayúsculas a minúsculas
* Se han eliminado las '@' de @USUARIO con el fin de facilitar el etiquetado morfológico
* Quitar los links 
* Quitar los emojis
* Eliminar las stopwords
* Se han reemplazado todos los números por el símbolo '0'
* Quitar los signos de puntuación y quitar espacios (tabuladores, etc)

In [4]:
pattern_URL="(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})"

def procesar(file, namefile):    
    file[file.columns[1]] = [clean_text(i) for i in file[file.columns[1]]]    
    file.to_csv(namefile, sep='\t', encoding='utf-8', index=False)
    return file
    
def clean_text(text):
    text = text.lower()   
    #text=re.sub("@([A-Za-z0-9_]{1,15})", "@USUARIO", text)
    text=re.sub("@([A-Za-z0-9_]{1,15})", " ", text)
    text=re.sub(pattern_URL, " ", text)
    
    text= remove_emoji(text)
    text= remove_stopwords(text)
    text=re.sub("\d+", "0", text)
    # text=re.sub("\d+", " ", text)
    
    text=re.sub(r" +", " ", re.sub(r"\t", " ", re.sub(r"\n+", "\n", re.sub('(?:[.,\/!$%?¿?!¡\^&\*;:{}=><\-_`~()”“"\'\|])', " ",text))))
    text = text.strip()
    return text

def remove_stopwords(text):    
    stopwords=set(nltk.corpus.stopwords.words("spanish"))
    for i in stopwords:
        text = re.sub(r"\b%s\b" % i, " ", text)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # emoticons
                               "\U0001F300-\U0001F5FF"  # symbols & pictographs                               
                               "\U0001F680-\U0001F6FF"  # transport & map symbols
                               "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "\U00002702-\U000027B0"
                               "\U000024C2-\U0001F251"
                               "\U0001f926-\U0001f937"
                               "\u200d"
                               "\u2640-\u2642"
                               "\U0001F1F2-\U0001F1F4"  # Macau flag
                               "\U0001F1E6-\U0001F1FF"  # flags
                               "\U0001F600-\U0001F64F"
                               "\U0001F1F2"
                               "\U0001F1F4"
                               "\U0001F620"
                               "]+", flags=re.UNICODE)   
    text = emoji_pattern.sub(r'', text) # no emoji
    return text

### Guardando el corpus ya procesado A

In [5]:
corpus_train_esA = procesar(corpus_train_esA, "train_es_cleanA.tsv")
corpus_dev_esA = procesar(corpus_dev_esA, "dev_es_cleanA.tsv")

In [6]:
corpus_train_esA

,id,text,HS,TR,AG
0,20001,easyjet quiere duplicar número mujeres piloto ...,1,0,0
1,20002,gobierno debe crear control estricto inmigraci...,1,0,0
2,20003,veo mujeres destruidas acoso laboral callejero...,0,0,0
3,20004,— respetuoso demás sólamente recuerdo si escor...,0,0,0
4,20007,antonio caballero ser mal gusto ignorante vez ...,0,0,0
...,...,...,...,...,...
4464,24996,cállateeee zorra ahre #cnco #bestboyband #ihea...,1,1,1
4465,24997,presidente cúcuta problemas venezolanos disput...,1,0,1
4466,24998,callaté visto dejo puta,0,0,0
4467,24999,hombres casan mujeres cabras saben fregar platos,1,0,0


In [7]:
#Leyendo el corpus ya procesado-limpio A

train_idA = corpus_train_esA[corpus_train_esA.columns[0]]
X_train_textA = corpus_train_esA[corpus_train_esA.columns[1]].fillna(' ')
y_train_hsA = corpus_train_esA[corpus_train_esA.columns[2]]

test_idA = corpus_dev_esA[corpus_train_esA.columns[0]]
X_test_textA = corpus_dev_esA[corpus_dev_esA.columns[1]].fillna(' ')
y_test_hsA = corpus_dev_esA[corpus_dev_esA.columns[2]]

In [8]:
print( len(X_train_textA), len(y_train_hsA) )

print( len(X_test_textA), len(y_test_hsA) )

4469 4469
500 500


Aplicando los CountVectorizer para construir la matriz termino 

In [9]:
cvectorizer = CountVectorizer(
    # lowercase=True,
    #stop_words=[word.decode('utf-8') for word in nltk.corpus.stopwords.words('spanish')],
    #token_pattern=r'\b\w+\b', #selects tokens of 2 or more alphanumeric characters 
    ngram_range=(1,3),#n-grams de palabras n = 1 a n = 3 (unigramas, bigramas y trigramas)
    min_df=5,#ignorando los términos que tienen una frecuencia de documento estrictamente inferior a 5
).fit(X_train_textA)

X_train_cvectorized = cvectorizer.transform(X_train_textA).toarray()
print(X_train_cvectorized.shape)

X_test_cvectorized = cvectorizer.transform(X_test_textA).toarray()
print(X_test_cvectorized.shape)

(4469, 1775)
(500, 1775)


In [10]:
tvectorizer = TfidfVectorizer(
    # lowercase=True,
    #stop_words=[word.decode('utf-8') for word in nltk.corpus.stopwords.words('spanish')],
    #token_pattern=r'\b\w+\b', #selects tokens of 2 or more alphanumeric characters 
    ngram_range=(1,3),#n-grams de palabras n = 1 a n = 3 (unigramas, bigramas y trigramas)
    min_df=5,#ignorando los términos que tienen una frecuencia de documento estrictamente inferior a 5
).fit(X_train_textA)

X_train_tvectorized = tvectorizer.transform(X_train_textA).toarray()
print(X_train_tvectorized.shape)

X_test_tvectorized = tvectorizer.transform(X_test_textA).toarray()
print(X_test_tvectorized.shape)

(4469, 1775)
(500, 1775)


   ### Perceptrón
   
   https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html

### Perceptrón Multicapa (Multi-Layer Perceptron, MLP) 

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

In [11]:
from sklearn.neural_network import MLPClassifier

mlp1 = MLPClassifier(hidden_layer_sizes=(10,10,10), max_iter=500, alpha=0.0001,
                    solver='adam', random_state=21,tol=0.000000001)
mlp2 = MLPClassifier(hidden_layer_sizes=(6,6,6,6),solver='lbfgs',max_iter=6000)

In [12]:
mlp1.fit( X_train_cvectorized, y_train_hsA)
predictions1 = mlp1.predict(X_test_cvectorized)

print('\t', 'Accuracy mlp1 cv', accuracy_score(y_test_hsA, predictions1))
print('\t', 'F1-score mlp1 cv', f1_score(y_test_hsA, predictions1))

######

mlp1.fit( X_train_tvectorized, y_train_hsA)
predictions1 = mlp1.predict(X_test_tvectorized)

print('\t', 'Accuracy mlp1 tfidv', accuracy_score(y_test_hsA, predictions1))
print('\t', 'F1-score mlp1 tfidv', f1_score(y_test_hsA, predictions1))

	 Accuracy mlp1 cv 0.756
	 F1-score mlp1 cv 0.7288888888888888
	 Accuracy mlp1 tfidv 0.74
	 F1-score mlp1 tfidv 0.7045454545454545


In [13]:
mlp2.fit( X_train_cvectorized, y_train_hsA)
predictions2 = mlp2.predict(X_test_cvectorized)

print('\t', 'Accuracy mlp2 cv', accuracy_score(y_test_hsA, predictions2))
print('\t', 'F1-score mlp2 cv', f1_score(y_test_hsA, predictions2))

######

mlp2.fit( X_train_tvectorized, y_train_hsA)
predictions2 = mlp2.predict(X_test_tvectorized)

print('\t', 'Accuracy mlp2 tfidv', accuracy_score(y_test_hsA, predictions2))
print('\t', 'F1-score mlp2 tfidv', f1_score(y_test_hsA, predictions2))

	 Accuracy mlp2 cv 0.742
	 F1-score mlp2 cv 0.7139689578713968
	 Accuracy mlp2 tfidv 0.754
	 F1-score mlp2 tfidv 0.7421383647798743


Las RN son sensibles a la escala de los datos de entrada, especialmente cuando se utilizan las funciones de activación sigmoide (por defecto) o tanh. Puede ser una buena práctica reescalar los datos al rango de 0 a 1, también llamado normalización. Podemos normalizar fácilmente el conjunto de datos utilizando la clase de preprocesamiento MinMaxScaler de la biblioteca scikit-learn.

In [14]:
# normalizacion de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train_cvectorized)
X_test = scaler.fit_transform(X_test_cvectorized)

In [15]:
mlp2.fit( X_train, y_train_hsA)
predictions2 = mlp2.predict(X_test)

print('\t', 'Accuracy mlp2', accuracy_score(y_test_hsA, predictions2))
print('\t', 'F1-score mlp2', f1_score(y_test_hsA, predictions2))

	 Accuracy mlp2 0.714
	 F1-score mlp2 0.6697459584295613


## Redes neuronales recurrentes 

<b>¿Qué es Keras?</b>

Keras es una biblioteca de Python minimalista para Deep Learning que puede funcionar sobre Theano o TensorFlow. Fue desarrollada con el objetivo de que los modelos de Deep Learning sean tan rápidos y fáciles tanto para la investigación como el desarrollo. Funciona en Python 2.7 o 3.6 y se puede ejecutar sin problemas sobre las GPU y las CPU. Es libre bajo una licencia del MIT. Keras fue desarrollado por Francois Chollet, un ingeniero de Google que utiliza cuatro principios rectores:
<ul>
    <li><b>Modularidad</b>: Un modelo puede entenderse sólo como una secuencia o como un gráfico. Todas las características de un modelo de aprendizaje profundo son componentes discretos que pueden combinarse de manera arbitraria.</li>
    <li><b>Minimalismo</b>: La biblioteca proporciona lo justo para lograr un resultado, sin florituras y maximizando la legibilidad.</li>
    <li><b>Extensibilidad</b>: Los nuevos componentes son intencionalmente fáciles de añadir y usar dentro del marco destinado a que los desarrolladores prueben y exploren nuevas ideas.</li>
    <li><b>Python</b>: No hay modelos separados con formatos personalizados. Todo es nativo de Python.</li>
</ul>


<b>Construya modelos de Deep Learning con Keras</b>

El enfoque de Keras es la idea de un modelo. El tipo principal de modelo es una secuencia de capas llamada Secuencial que es una pila lineal de capas. Se crea un Secuencial y se le añaden capas en el orden en que desea que se realice el cálculo. Una vez definido, se compila el modelo que utiliza el marco subyacente para optimizar el cálculo que se va a realizar. En este se puede especificar la función de pérdida y el optimizador a utilizar.

Una vez compilado, el modelo debe ajustarse a los datos. Esto se puede hacer con un lote de datos o por el anillo o el régimen de entrenamiento del modelo entero. Aquí es donde ocurre todo el cálculo. Una vez entrenado, puede usar su modelo para hacer predicciones sobre nuevos datos. Podemos resumir la construcción de modelos de Deep learning en Keras de la siguiente manera:
<ul>
    <li><b>Define tu modelo</b>. Cree un modelo secuencial y añada capas.</li>
    <li><b>Compila tu modelo</b>. Especifique la función de pérdida y los optimizadores y llame a compile() en el modelo.</li>
    <li><b>Ajuste a su modelo</b>. Entrene el modelo sobre una muestra de datos llamando a la función fit() en el modelo.</li>
    <li><b>Haga predicciones</b>. Utilice el modelo para generar predicciones sobre nuevos datos llamando a evaluate() o predict().</li>
</ul>

In [8]:
print( X_train_cvectorized.shape, len(y_train_hsA), 'Secuencia de entrenamiento' )

print( X_test_cvectorized.shape, len(y_test_hsA), 'Secuencia de prueba' )

(4469, 1775) 4469 Secuencia de entrenamiento
(500, 1775) 500 Secuencia de prueba


In [16]:
from keras.models import Sequential
 
model = Sequential()

In [17]:
# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

# La clase layer de redes RNN
from keras.layers import Embedding, SimpleRNN

# Como cualquier otra layer de Keras, SimpleRNN procesa lotes de secuencias Numpy.
# La entrada es de la forma (batch_size, timesteps, input_features) en vez de (timesteps, input_features).
# [muestras, pasos de tiempo, características]

from keras.layers import Dense

max_features = 10000  # tamaño del diccionario de palabras comunes
                      # (número de palabras a utilizar)
maxlen = 1775         # longitud máxima de cada secuencia 
batch_size = 32

# Capa embedding
# input_dim : tamaño del vocabulario
# output_dim: dimensión del vector al que se mapea
#Se usa un embedding con tamaño de diccionario a los más de 10,000 y se mapean a dimensión un vector de dimensión 32
# Las Embedding (incrustaciones) son una excelente manera de lidiar con los problemas de PNL por dos razones. 
# Primero, ayuda en la reducción de la dimensionalidad sobre la codificación one-hot, ya que podemos controlar la 
# cantidad de características. En segundo lugar, es capaz de comprender el contexto de una palabra para que
# palabras similares tengan incrustaciones similares.
model.add(Embedding(input_dim=max_features, output_dim=32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))
# RNN con una capa Embedding y una capa SimpleRNN que regresa solo una salida para cada secuencia

# Resumen de la arquitectura
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 322113 (1.23 MB)
Trainable params: 322113 (1.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
model.compile(
    optimizer='rmsprop', # algoritmo de optimización que se utiliza durante el entrenamiento del modelo. En este caso, 'rmsprop' se refiere a "Root Mean Square Propagation", que es un algoritmo de optimización popular en el campo del aprendizaje profundo.
    loss='binary_crossentropy', # función de pérdida (loss function) utilizada durante el entrenamiento del modelo
    # En particular, binary_crossentropy se utiliza en problemas de clasificación binaria
    # Mide la discrepancia entre las probabilidades predichas por el modelo y las etiquetas reales (objetivos) de los datos de entrenamiento.
    # Cuanto menor sea el valor de la pérdida, mejor será el ajuste del modelo a los datos de entrenamiento
    metrics=['acc']
)
# optimizer='rmsprop' significa que durante el entrenamiento del modelo de secuencia, se utilizará el algoritmo de optimización 
# RMSprop para ajustar los pesos y sesgos de la red neuronal con el objetivo de minimizar la función de pérdida y mejorar el rendimiento del modelo.
# loss='binary_crossentropy' se utiliza en modelos de secuencia para problemas de clasificación binaria, y es una medida
# de cuán bien el modelo se ajusta a los datos de entrenamiento en términos de la probabilidad de las etiquetas predichas en comparación con las etiquetas reales.

import time
tic = time.time()
history = model.fit(
    X_train_cvectorized, y_train_hsA,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)
print('Tiempo de entrenamiento:', time.time()-tic)

Epoch 1/10
28/28 - 41s - loss: 0.6781 - acc: 0.5768 - val_loss: 0.6749 - val_acc: 0.5783 - 41s/epoch - 1s/step
Epoch 2/10
28/28 - 34s - loss: 0.6695 - acc: 0.5905 - val_loss: 0.6706 - val_acc: 0.5783 - 34s/epoch - 1s/step
Epoch 3/10
28/28 - 35s - loss: 0.6642 - acc: 0.6173 - val_loss: 0.6600 - val_acc: 0.6286 - 35s/epoch - 1s/step
Epoch 4/10
28/28 - 30s - loss: 0.6667 - acc: 0.6283 - val_loss: 0.6603 - val_acc: 0.6298 - 30s/epoch - 1s/step
Epoch 5/10
28/28 - 30s - loss: 0.6601 - acc: 0.6238 - val_loss: 0.6585 - val_acc: 0.6320 - 30s/epoch - 1s/step
Epoch 6/10
28/28 - 29s - loss: 0.6528 - acc: 0.6369 - val_loss: 0.6522 - val_acc: 0.6275 - 29s/epoch - 1s/step
Epoch 7/10
28/28 - 22s - loss: 0.6522 - acc: 0.6297 - val_loss: 0.6517 - val_acc: 0.6286 - 22s/epoch - 798ms/step
Epoch 8/10
28/28 - 23s - loss: 0.6536 - acc: 0.6249 - val_loss: 0.6519 - val_acc: 0.6298 - 23s/epoch - 810ms/step
Epoch 9/10
28/28 - 22s - loss: 0.6476 - acc: 0.6347 - val_loss: 0.6474 - val_acc: 0.6309 - 22s/epoch - 783

In [19]:
# evaluate the model
scores = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# La función model.evaluate predice la salida para la entrada dada y luego calcula la función de métrica 
# especificada en model.compile y basada en y_true y y_pred y devuelve el valor de métrica calculada como salida

# make predictions
testPredict = model.predict(X_test_cvectorized)
# model.predict simplemente devuelve el y_pred
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict.round()))

# si usamos model.predict y luego calculamos las métricas uno mismo, el valor de la métrica calculada 
# debería resultar ser el mismo que model.evaluate

acc: 61.80%
16/16 [==============================] - 3s 120ms/step
	 Accuracy 0.618


### Ejemplo con Stack de RNNs

In [20]:
# Ejemplo con Stack de RNNs

model = Sequential()
# Capa embedding
# input_dim : tamaño del vocabulario
# output_dim: dimensión del vector al que se mapea
model.add(Embedding(input_dim=max_features, output_dim=32))
model.add(SimpleRNN(32, return_sequences=True)) # Si devolver la última salida en la secuencia de salida o la secuencia completa
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(
    optimizer='rmsprop', 
    loss='binary_crossentropy',
    metrics=['acc']
)

tic = time.time()
history_stackRNN = model.fit(
    X_train_cvectorized, y_train_hsA,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)
print('Tiempo de entrenamiento:', time.time()-tic)

# evaluate the model
scores = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# make predictions
testPredict_stackRNN = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_stackRNN.round()))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          320000    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 324193 (1.24 MB)
Trainable params: 324193 (1.24 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
28/28 - 82s - loss: 0.6746 - acc: 0.5952 - val_loss: 0.6507 - val_acc: 0.6298 - 82s/epoch - 3s/step
Epoch 2/10
28/28 - 66s - loss: 0.6546 -

### Red de Memoria Corta a Largo Plazo (Long-Term Short Memory, LTSM)

In [22]:
from keras.layers import LSTM

print('Número máximo de palabras a usar:', max_features)

# Creamos el modelo con la incrustación (embedding)

model = Sequential()
model.add(Embedding(max_features, 32))

# Incluimos una capa LSTM
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

# Parámetros de entrenamiento
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
)

# y entrenamos
tic=time.time()
history_LSTM = model.fit(
    X_train_cvectorized, y_train_hsA,
    epochs=6,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)
print('tiempo de entrenamiento: ', time.time()-tic)

# evaluate the model
scores = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# make predictions
testPredict_LSTM = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_LSTM.round()))

Número máximo de palabras a usar: 10000
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________
Train on 3575 samples, validate on 894 samples
Epoch 1/6
 - 28s - loss: 0.6812 - acc: 0.5801 - val_loss: 0.6807 - val_acc: 0.5783
Epoch 2/6
 - 29s - loss: 0.6771 - acc: 0.5913 - val_loss: 0.6808 - val_acc: 0.5783
Epoch 3/6
 - 31s - loss: 0.6769 - acc: 0.5913 - val_loss: 0.6812 - val_acc: 0.5783
Epoch 4/6
 - 30s - loss: 0.6773 - acc: 0.5

### Redes con Unidades Recurrentes con Compuertas (Gated-RU)

In [23]:
from keras.layers import GRU

model = Sequential()
# Capa embedding
# input_dim : tamaño del vocabulario
# output_dim: dimensión del vector al que se mapea

model.add(Embedding(input_dim=max_features, output_dim=32))
# comentar la siguiente linea para evaluar dropout 
model.add(GRU(32))
# descomentar la siguiente linea para evaluar dropout 
#model.add(GRU(32, dropout=.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['acc'])

tic = time.time()
history_GRU = model.fit(
    X_train_cvectorized, y_train_hsA,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)
print('Tiempo de entrenamiento:', time.time()-tic)

# evaluate the model
scores = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# make predictions
testPredict_GRU = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_GRU.round()))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 326,273
Trainable params: 326,273
Non-trainable params: 0
_________________________________________________________________
Train on 3575 samples, validate on 894 samples
Epoch 1/10
 - 31s - loss: 0.6795 - acc: 0.5841 - val_loss: 0.6810 - val_acc: 0.5783
Epoch 2/10
 - 35s - loss: 0.6770 - acc: 0.5913 - val_loss: 0.6813 - val_acc: 0.5783
Epoch 3/10
 - 39s - loss: 0.6767 - acc: 0.5913 - val_loss: 0.6810 - val_acc: 0.5783
Epoch 4/10
 - 40s - loss: 0.6768 - acc: 0.5913 - val_loss: 0.6809 - val_acc: 0.

In [11]:
# Ejemplo con Stack de RNNs

# La clase layer de redes RNN
from keras.layers import Embedding, SimpleRNN

# Como cualquier otra layer de Keras, SimpleRNN procesa lotes de secuencias Numpy.
# La entrada es de la forma (batch_size, timesteps, input_features) en vez de (timesteps, input_features).
# [muestras, pasos de tiempo, características]

from keras.layers import Dense

max_features = 10000  # tamaño del diccionario de palabras comunes
                      # (número de palabras a utilizar)
maxlen = 1775         # longitud máxima de cada secuencia 
batch_size = 32


model = Sequential()
# Capa embedding
# input_dim : tamaño del vocabulario
# output_dim: dimensión del vector al que se mapea
model.add(Embedding(input_dim=max_features, output_dim=32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(
    optimizer='rmsprop', 
    loss='binary_crossentropy',
    metrics=['acc']
)

import time
tic = time.time()
history_stackRNN = model.fit(
    X_train, y_train_hsA,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)
print('Tiempo de entrenamiento:', time.time()-tic)

# evaluate the model
scores = model.evaluate(X_test, y_test_hsA, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# make predictions
testPredict_stackRNN = model.predict(X_test)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_stackRNN.round()))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 324,193
Trainable params: 324,193
Non-trainable params: 0
_________________________________________________________________

Train on 3575 samples, validate on 894 samples
Epoch 1/10
 - 30s - loss: 0.6902 - acc: 0.5676 - val_loss: 0.6819 - val_acc: 0.5783
Epoch 2/10
 - 29s - loss: 0.6797 - acc: 0.5910 - val_loss: 0.6914 - val_acc: 0.5783
Epoch 3/10
 - 26s - loss: 0.67

### Modelos basados en atención y transformers

In [ ]:
https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
    
    https://towardsdatascience.com/recurrent-neural-networks-by-example-in-python-ffd204f99470
        
        http://personal.cimat.mx:8181/~mrivera/cursos/aprendizaje_profundo/RNN_LTSM/introduccion_rnn.html
                
                https://unipython.com/prediccion-con-series-temporales-con-lstm-redes-neuronales-recurrentes/